In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

import math
from collections import Counter

import pickle
import pydotplus

def openCsv(filename):
    df = pd.read_csv(filename)
    data = np.array(df)
    np.random.shuffle(data)
    allPos = data[data[:, -1]==1] # 492
    allNeg = data[data[:, -1]==0] # 284315
    data = np.r_[allPos, allNeg[:508]]
    np.random.shuffle(data)
    return data[:, 1:-1], data[:, -1]

all_xs, all_ys = openCsv('./creditcard.csv')
train_xs, test_xs, train_ys, test_ys = train_test_split(all_xs, all_ys, test_size=0.2, shuffle=True)

train_ys = train_ys.astype(np.int)
test_ys = test_ys.astype(np.int)
col_name = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount"]

print(f'train: {Counter(train_ys)}, test: {Counter(test_ys)}')

train: Counter({0: 404, 1: 396}), test: Counter({0: 104, 1: 96})


In [8]:
class Node:
    '''
    attr 属性值 split 分割值 pos 正数 neg 负数 classLabel 最终正负类别
    '''
    def __init__(self, attr='', split=0, pos=0, neg=0):
        self.attr = attr
        self.split = split
        self.pos = pos
        self.neg = neg
        self.classLabel = '+' if pos >= neg else '-'
        
    def __str__(self):
        if self.attr == '':
            return '[{}, {}]: {}'.format(self.neg, self.pos, self.classLabel)
        return '{}<={}\n[{}, {}]: {}'.format(self.attr, self.split, self.neg, self.pos, self.classLabel)

class Tree:
    def __init__(self, curr='', left=None, right=None):
        self.curr = curr
        self.left = left
        self.right = right
        
    def __str__(self):
        if self.curr == None:
            return ''
        return '{} | {} | {}'.format(str(self.curr).replace('\n', ' '), self.left, self.right)

$$ \text{Gini} = \sum^{K}_{k=1}p_k(1-p_k) = 1-\sum^{K}_{k=1}p_k^2 $$
$$ \text{Gini}_2 = 1 - [p_+^2 + (1 - p_+)^2] = 2p(1-p) $$
$$ \text{Gini}_K = 1-\sum^{K}_{k=1}(\frac{|C_k|}{|D|})^2 $$

$$ \text{Gini}(D, \text{Attr}_M) = \sum^{M}_{m=1}\frac{D_m}{D}\text{Gini}(D_m) $$
$$ \text{Gini}(D, \text{Attr}_2) = \frac{D_l}{D}\text{Gini}(D_l) + \frac{D_r}{D}\text{Gini}(D_r) $$
$$ \text{Split_Attr}(D) = \arg\min_{\text{Attr}} \text{Gini}(D, \text{Attr})\quad (\text{Attr}\in \text{Attrs}) $$

In [3]:
def calcTwoClassGini(labels: []) -> float:
    '''
    对 0, 1 列表计算基尼系数
    '''
    pr = (np.sum(labels) / len(labels)) if len(labels) != 0 else 0
    return 2 * pr * (1 - pr)

def getMinGiniCol(xs: [[]], ys: []) -> (int, float):
    '''
    计算矩阵内的最优特征(最小基尼系数)与分割值
    '''
    def calcOneColGini(one_attr_xs: [float], ys: [int]) -> (float, float, float):
        '''
        在一列连续性 xs 中 取最佳分割中位数
        :return: split, gain
        '''
        gini = calcTwoClassGini(ys) # 整个数据集的Gini
        one_attr_sorted_xs = np.unique(one_attr_xs) # 去重并排序
        medians = ((np.r_[0, one_attr_sorted_xs] + np.r_[one_attr_sorted_xs, 0]) / 2) [1:-1]
        
        minGini = math.inf  # 最小的Gini
        goalMedian = 0  # 目标的中位数分割节点
        for median in medians:
            idxLeft = one_attr_xs <= median
            ysLeft = ys[idxLeft]
            ysRight = ys[~idxLeft]
            
            giniLeft = calcTwoClassGini(ysLeft) # 第一个区间的Gini
            giniRight = calcTwoClassGini(ysRight) # 第二个区间的Gini
            pr = len(ysLeft) / len(ys) # 第一个区间占整个数据的比例
            attr_jini = gini - (pr * giniLeft + (1 - pr) * giniRight) # 用 median 中位数拆分的 Gini
            if minGini > attr_jini:
                minGini, goalMedian = attr_jini, median
                
        return goalMedian, minGini
        
    minGini = math.inf # 最小的Gini
    goalColIdx, goalSplit = 0, 0  # 目标的最优特征列和分割值
    for idx, idx in enumerate(range(xs.shape[1])):
        gini, split = calcOneColGini(xs[:, idx], ys)
        if minGini > gini:
            minGini = gini
            goalColIdx, goalSplit = idx, split
    
    return goalColIdx, goalSplit

In [15]:
maxDepth = math.inf

def splitTree(data: [[]], ys: [], decisionTree, depth=0):
    colIdx, split = getMinGiniCol(data, ys) # 目标列与分割点
    print('Greatest col: \"{}\", split: {}'.format(col_name[colIdx], split))
    
    leftIdxs = data[:, colIdx] <= split
    rightIdxs = ~leftIdxs
    
    left, leftYs = data[leftIdxs], ys[leftIdxs] # 第一棵树的数据与标签
    right, rightYs = data[rightIdxs], ys[rightIdxs] # 第二棵树的数据与标签
    
    allCounter = Counter(ys)
    print('all: ', allCounter)
    decisionTree.curr = Node(col_name[colIdx], round(split, 3), pos=allCounter[1], neg=allCounter[0]) # 当前节点：属性名，分裂值，正负类数
    
    leftCounter = Counter(leftYs)
    rightCounter = Counter(rightYs)
    if len(leftCounter) == 0 or len(rightCounter) == 0: # 没有分类到
        return
    
    if depth < maxDepth: # 在层数内 
        print('left: ', leftCounter, len(leftCounter))
        if len(leftCounter) == 1: # 只有一类
            decisionTree.left = Tree(Node(pos=leftCounter[1], neg=leftCounter[0])) # 左子树：输出正负
        else:
            decisionTree.left = Tree()
            splitTree(left, leftYs, decisionTree.left, depth + 1) # 分割左子树

        print('right: ', rightCounter, len(rightCounter))
        if len(rightCounter) == 1: # 只有一类
            decisionTree.right = Tree(Node(pos=rightCounter[1], neg=rightCounter[0])) # 右子树：输出正负
        elif depth < maxDepth:
            decisionTree.right = Tree()
            splitTree(right, rightYs, decisionTree.right, depth + 1) # 分割右子树
        
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

decisionTree = Tree('data')
splitTree(train_xs, train_ys, decisionTree)

print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

2019-11-30 13:13:12
Greatest col: "V2", split: 1.2531328325016133e-07
all:  Counter({0: 404, 1: 396})
Counter({0: 207, 1: 45}) Counter({1: 351, 0: 197})
left:  Counter({0: 207, 1: 45}) 2
Greatest col: "V1", split: 0.0002540856980242645
all:  Counter({0: 207, 1: 45})
Counter({0: 57, 1: 36}) Counter({0: 150, 1: 9})
left:  Counter({0: 57, 1: 36}) 2
Greatest col: "V2", split: 2.0011206274817006e-06
all:  Counter({0: 57, 1: 36})
Counter({0: 57, 1: 36}) Counter()
right:  Counter({0: 150, 1: 9}) 2
Greatest col: "V15", split: 4.0556799798113774e-05
all:  Counter({0: 150, 1: 9})
Counter({0: 79, 1: 8}) Counter({0: 71, 1: 1})
left:  Counter({0: 79, 1: 8}) 2
Greatest col: "V10", split: 0.00019664051962256512
all:  Counter({0: 79, 1: 8})
Counter({0: 25, 1: 4}) Counter({0: 54, 1: 4})
left:  Counter({0: 25, 1: 4}) 2
Greatest col: "V5", split: 0.0013589264481059515
all:  Counter({0: 25, 1: 4})
Counter({0: 22, 1: 1}) Counter({0: 3, 1: 3})
left:  Counter({0: 22, 1: 1}) 2
Greatest col: "V15", split: 0.00

In [6]:
def saveTree(root, filename):
    with open(filename, 'wb') as f:
        f.write(pickle.dumps(root))
        
def readTree(filename):
    obj = Tree()
    with open(filename, 'rb') as f:
        obj = pickle.loads(f.read())
    return obj

# decisionTree = readTree('./decisionTree_C4_5.pkl')

In [11]:
print(str(decisionTree))

def toDot(root):
    def genNode(idx, label):
        return f'{idx} [label="{label}" fontsize=8] ;\n'
    def genLink(idx1, idx2, label, isLeft=True):
        return f'{idx1} -> {idx2} [labeldistance=2.5, labelangle={45 if isLeft else -45}, headlabel="{label}"] ;\n'
    def genHead(shape='box'): # ellipse
        return f'digraph Tree {{ \nnode [shape={shape}] ;\n'
    def genTail():
        return '}\n'
    
    def genBody(dot, idx, node, isRoot=False):
        dot += genNode(idx, node.curr)
        newIdx = idx
        if node.left != None:
            dot += genLink(idx, idx + 1, 'True' if isRoot else '', isLeft=True)
            dot, newIdx = genBody(dot, idx + 1, node.left)
        if node.right != None:
            dot += genLink(idx, newIdx + 1, 'False' if isRoot else '', isLeft=False)
            dot, newIdx = genBody(dot, newIdx + 1, node.right)
        return dot, newIdx
    
    dot = genHead()
    dot += genBody('', 0, root, isRoot=True)[0]
    dot += genTail()
    return dot

saveTree(decisionTree, 'decisionTree_CART.pkl')
dot = toDot(decisionTree)
graph = pydotplus.graph_from_dot_data(dot)
graph.write_pdf("decisionTree_CART.pdf")

V2<=0.0 [404, 396]: - | V1<=0.0 [207, 45]: - | V2<=0.0 [57, 36]: - | None | None | V15<=0.0 [150, 9]: - | V10<=0.0 [79, 8]: - | V5<=0.001 [25, 4]: - | V15<=0.0 [22, 1]: - | None | None | V3<=0.056 [3, 3]: + | None | None | V12<=0.0 [54, 4]: - | V11<=0.001 [25, 3]: - | V3<=0.001 [14, 1]: - | V3<=0.003 [8, 1]: - | None | None | [6, 0]: - | None | None | V2<=0.004 [11, 2]: - | None | None | V3<=0.0 [29, 1]: - | [20, 0]: - | None | None | V11<=0.002 [9, 1]: - | V11<=0.011 [5, 1]: - | None | None | [4, 0]: - | None | None | V3<=0.0 [71, 1]: - | V3<=0.0 [50, 1]: - | None | None | [21, 0]: - | None | None | V3<=0.001 [197, 351]: + | V3<=0.0 [75, 334]: + | None | None | V1<=0.0 [122, 17]: - | V8<=0.0 [94, 12]: - | V4<=0.001 [40, 9]: - | V8<=0.0 [30, 1]: - | None | None | V22<=0.001 [10, 8]: - | V1<=0.0 [6, 3]: - | None | None | V5<=0.049 [4, 5]: + | [0, 3]: + | None | None | V15<=0.044 [4, 2]: - | [2, 0]: - | None | None | V14<=0.0 [2, 2]: + | None | None | V19<=0.0 [54, 3]: - | V19<=0.0 [27, 

True

In [16]:
def searchInTree(tree, x) -> int:
    if tree.curr != None and (tree.left is None or tree.right is None): # 叶子节点
        return tree.curr.classLabel == '+' # + -> 1, - -> 0
    idx = col_name.index(tree.curr.attr)
    if x[idx] <= tree.curr.split:
        return searchInTree(tree.left, x)
    else:
        return searchInTree(tree.right, x)

train_pos = len(list(filter(lambda y: y == 1, train_ys)))
train_neg = len(train_ys) - train_pos
test_pos = len(list(filter(lambda y: y == 1, test_ys)))
test_neg = len(test_ys) - test_pos

pred = np.zeros(len(train_ys))
print('train pos: {}, neg: {}, acc(all neg): {:.6f}, f1(all neg): {:.6f}'.format(train_pos, train_neg, accuracy_score(train_ys, pred), f1_score(train_ys, pred)))
pred = np.zeros(len(test_ys))
print('test  pos: {}, neg: {}, acc(all neg): {:.6f}, f1(all neg): {:.6f}'.format(test_pos, test_neg, accuracy_score(test_ys, pred), f1_score(test_ys, pred)))

pred = [searchInTree(decisionTree, x) for x in train_xs]
print('train acc: {:.6f}, f1: {:.6f}'.format(accuracy_score(train_ys, pred), f1_score(train_ys, pred)))
pred = [searchInTree(decisionTree, x) for x in test_xs]
print('test  acc: {:.6f}, f1: {:.6f}'.format(accuracy_score(test_ys, pred), f1_score(test_ys, pred)))

train pos: 396, neg: 404, acc(all neg): 0.505000, f1(all neg): 0.000000
test  pos: 96, neg: 104, acc(all neg): 0.520000, f1(all neg): 0.000000
train acc: 0.832500, f1: 0.836186
test  acc: 0.840000, f1: 0.840000


e:\program files (x86)\python\python36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
_decisionTree = decisionTree

In [17]:
def postPruning(root, node, xs, ys):
    """
    对决策树进行后剪枝，从叶子节点往上遍历
    """
    if node.curr == None:
        return
    if not (node.left.left is None or node.left.right is None or node.right.left is None or node.right.right is None): # 子节点非叶子
        postPruning(root, node.left, xs, ys) # 子节点遍历，后序
        postPruning(root, node.right, xs, ys)
    else: # 子节点为叶子 
        # 未剪枝
        pred = [searchInTree(root, x) for x in xs]
        acc_not_pruning = accuracy_score(ys, pred)
    
        # 剪枝
        saveLeft, saveRight = node.left, node.right
        node.left, node.right = None, None
        pred = [searchInTree(root, x) for x in xs]
        acc_pruning = accuracy_score(ys, pred)
        
        if acc_pruning < acc_not_pruning: # 精确率降低
            node.left, node.right = saveLeft, saveRight
            print(f'Not Prune: {node.curr.attr} <= {node.curr.split} | not prune: {acc_not_pruning} -> prune: {acc_pruning}')
        else:
            print(f'Prune: {node.curr.attr} <= {node.curr.split} | not prune: {acc_not_pruning} -> prune: {acc_pruning}')
    
postPruning(decisionTree, decisionTree, train_xs, train_ys)

Prune: V1 <= 0.0 | not prune: 0.8325 -> prune: 0.8325
Not Prune: V3 <= 0.001 | not prune: 0.8325 -> prune: 0.6975


In [18]:
saveTree(decisionTree, 'decisionTree_postPruning_CART.pkl')
dot = toDot(decisionTree)
graph = pydotplus.graph_from_dot_data(dot)
graph.write_pdf("decisionTree_postPruning_CART.pdf")

pred = [searchInTree(decisionTree, x) for x in train_xs]
print('train acc: {:.6f}, f1: {:.6f}'.format(accuracy_score(train_ys, pred), f1_score(train_ys, pred)))
pred = [searchInTree(decisionTree, x) for x in test_xs]
print('test  acc: {:.6f}, f1: {:.6f}'.format(accuracy_score(test_ys, pred), f1_score(test_ys, pred)))

train acc: 0.832500, f1: 0.834975
test  acc: 0.840000, f1: 0.840000


In [19]:
[(searchInTree(decisionTree, x), y) for x, y in zip(test_xs[0:20], test_ys[0:20])]

[(True, 0),
 (False, 0),
 (False, 1),
 (False, 0),
 (False, 0),
 (False, 0),
 (True, 1),
 (True, 1),
 (False, 0),
 (True, 1),
 (False, 0),
 (False, 1),
 (True, 0),
 (True, 1),
 (False, 1),
 (False, 0),
 (True, 1),
 (True, 1),
 (True, 1),
 (False, 0)]